<a href="https://colab.research.google.com/github/said3427/Tools/blob/master/cooperation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install p_tqdm

     |████████████████████████████████| 81 kB 5.7 MB/s 
     |████████████████████████████████| 65 kB 2.5 MB/s 
  Created wheel for p-tqdm: filename=p_tqdm-1.3.3-py3-none-any.whl size=3987 sha256=4b8b01f1f5df5fc26bd284bd824762255ed0d7be65cde855a2e993e4451a5c31
  Stored in directory: /root/.cache/pip/wheels/57/6c/d6/8c4cc7d253ecfdfe0fb49f5a754d33e9c2ce1664935325b1b6
Successfully built p-tqdm


In [54]:
import numpy as np
import scipy
from scipy.stats import norm, uniform, multivariate_normal
from scipy.optimize import minimize
import sys, ast
from random import choices, seed, random
from tqdm import tqdm
from p_tqdm import p_umap  # parallel code
from functools import partial
import os
from scipy.integrate import odeint, solve_ivp


# Create folder for output
if os.path.isdir('smc') is False:  ## if 'smc' folder does not exist:
    os.mkdir('smc')  ## create it, the output will go there

    
    
# Experimental data to fit the model to
x_data = np.array([0, 2, 4, 24])  # input data, time points
c_data = np.array(271666666.67,1200000000.00,1766666666666.67,22666666666666.70
) # input data, population/10^6
s_data = np.array([0, 550, 1970, 91700]) # input data, population/10^6

print('Time points:', x_data)
print('Cheater data:', c_data)
print('Cooperator data:', s_data, '\n')



# List parameters and upper/lower limits
parlist = [  
    {'name': 'growthc', 'lower_limit': 0.2, 'upper_limit': 0.6},  # cheater growth rate at glucose saturation
    {'name': 'growths', 'lower_limit': 0.2, 'upper_limit': 0.6},  # secretor growth rate at glucose saturation
    {'name': 'dc', 'lower_limit': 5.0, 'upper_limit': 15.0},  # cheater death rate at ampicillin saturation
    {'name': 'ds', 'lower_limit': 5.0, 'upper_limit': 15.0},  # secretor death rate at ampicillin saturation
    {'name': 'KG', 'lower_limit': 20.0, 'upper_limit': 60.0},  # glucose concentration at half max growth rate
    {'name': 'KAC', 'lower_limit': 3.0, 'upper_limit': 5.0},  # ampicillin concentration at half max death rate
    {'name': 'KAS', 'lower_limit': 1.0, 'upper_limit': 3.0},  # ampicillin concentration at half max death rate
    {'name': 'hydrolysis', 'lower_limit': 15.000, 'upper_limit': 25.000},  # rate of beta-lactamase action
    {'name': 'da', 'lower_limit': 0000.01, 'upper_limit': 0000.01},  # ampicillin degradation rate
    {'name': 'de', 'lower_limit': 0000.01, 'upper_limit': 0000.01},  # enzyme degradation rate
    {'name': 'uptake', 'lower_limit': 10.0000, 'upper_limit': 20.0000},  # rate of glucose uptake at glucose saturation
    {'name': 'secretion', 'lower_limit': 0.005, 'upper_limit': 0.008},  # rate of enzyme secretion at glucose saturation
]
    
# Create a dictionary for the parameters to make code easier to read
# Transform an array of pars e.g. p[0],p[1],p[2] into a named dictionary e.g. p['k0'],p['B'],p['n']
def pars_to_dict(pars):
    dict_pars = {} # create empty dictionary
    for parindex, par in enumerate(parlist):
        dict_pars[par['name']] = pars[parindex]
        #print(par['name'], pars[parindex], '\n')
    return dict_pars



def model(t, x, pars):
    ### This is the function you are trying to fit, so given a set of parameters (pars)
    ### and a set of concentrations x of signal/inducer, it returns the predicted response
    ### For instance for a repressive hill function:
    p = pars_to_dict(pars)

    #let x be a vector of
    cheater,secretor,glucose,ampicillin,enzyme = x 
    #set all 5 equations
    dxdt = [p['growthc'] * (glucose/(p['KG']+glucose)) * cheater - p['dc'] * ampicillin/(p['KAC']+ ampicillin), #cheaters
            p['growths'] * (glucose/(p['KG']+glucose)) * secretor - p['ds'] * ampicillin/(p['KAS']+ ampicillin), #cooperators
            - p['uptake'] * (glucose/(p['KG']+glucose)) * (cheater+secretor), #glucose
            - p['hydrolysis'] * ampicillin * enzyme-p['da'] * ampicillin, #ampicillin
            p['secretion'] * glucose/(p['KG']+glucose) * secretor - p['de'] * enzyme #enzyme
            ]
    return dxdt


def get_trajectory(timespan, initcond, t_eval, pars):
    #print('get_traj pars:', pars)
    #print('\t+ode', end='')
    traj = solve_ivp(model, t_span=timespan, y0=initcond, t_eval=t_eval, args=pars)
    #print('-ode', end='\t')
    return traj


timespan = [0, 24]
t_eval=[0,2,4,24]

def distance(par):  
    ### This function defines the distance between the data and the model for a given set of parameters
    ### for instance it can be a least squares minimization e.g. for a set of experimental observations
    ### stored as xdata and ydata:
    par = (tuple(par),)
    traj = get_trajectory(timespan, initcond, t_eval, par) 
    print(np.power(c_data - traj.y[0], 2))
    print(np.power(s_data - traj.y[1], 2))
    difference = np.sum(np.power(c_data - traj.y[0], 2) + np.power(s_data - traj.y[1], 2))
    return difference


# For each parameter take one random value from a uniform distribution between its upper/lower limits
# These will be the priors for the first iteration
def sampleprior():
    prior = []
    for parindex, par in enumerate(parlist):
        prior.append(uniform.rvs(loc=par['lower_limit'], 
                                 scale=par['upper_limit']))
    return prior

print('Initial priors:', sampleprior())

# For each parameter value proposed by the script
# Calculate how probable it is given a uniform distribution between the upper/lower limits
def evaluateprior(pars, model=None):
    ### Given a set of parameters return how probable is given the prior knowledge.
    ### If the priors are uniform distributions, this means that it will return 0 when a parameter
    ### outside the prior range is tested.
    prior = 1
    for parindex, par in enumerate(parlist):
        prior *= uniform.pdf(pars[parindex], loc=par['lower_limit'],
                             scale=par['upper_limit'])
    return prior

def GeneratePar(processcall=0, previousparlist=None, previousweights=None,
                eps_dist=100000000, kernel=None):

    # Create a function called Generate Par 
    # This function generates a valid parameter given a set of sampled pars previousparlist,previousweights
    # that is under a threshold eps_dist

    # processcall is a dummy variable that can be useful when tracking the function performance
    # it also allows the use of p_tqdm mapping that forces the use of an iterator

    seed()  # setting random seeds for each thread/process to avoid having the same random sequence in each thread
    np.random.seed()

    evaluated_distances = []
    d = eps_dist + 1  # original distance is beyond eps_dist
    while d > eps_dist:  # until a suitable parameter is found
        if previousparlist is None:  # if is the first iteration
            proposed_pars = sampleprior()
        else:
            print('+ch', end='')
            selected_pars = choices(previousparlist, weights=previousweights, k=1)  # select a point from the previous sample
            print('-ch', end='')
            proposed_pars = selected_pars + kernel.rvs()  # and perturb it

        
        if (evaluateprior(proposed_pars, model) > 0):
            print('+epd', proposed_pars[0], end=' ')
            d = distance(proposed_pars)
            if d is None:
              pass
            evaluated_distances.append(d)
            print('-epd:', d)

    # Calculate weight
    if previousparlist is None:
        weight = 1
    else:
        sum_denom = 0
        for ipars, pars in enumerate(previousparlist):
            kernel_evaluation = kernel.pdf(proposed_pars - pars)
            sum_denom += kernel_evaluation * previousweights[ipars]
        weight = evaluateprior(proposed_pars) / sum_denom
        
    print('end of GeneratePar')
    return proposed_pars, d, weight, evaluated_distances



def GeneratePars(previousparlist=None, previousweights=None, eps_dist=10000000,
                 Npars=1000, kernelfactor=1.0):
    ## Call to function GeneratePar in parallel until Npars points are accepted

    if previousparlist is not None:
        previouscovar = 2.0 * kernelfactor * np.cov(np.array(previousparlist).T)
        # print('covariance matrix previous parset:',previouscovar)
        kernel = multivariate_normal(cov=previouscovar)

    else:
        kernel = None

    trials = 0

    # for N in tqdm(range(Npars)): ## not parallel version
    #   GenerateParstePar(0,model = 'Berg',gamma = gamma, previousparlist = previousparlist,
    #   previousweights = previousweights, eps_dist = eps_dist, kernel = kernel)

    results = p_umap(
    partial(GeneratePar, previousparlist=previousparlist,
                previousweights=previousweights, eps_dist=eps_dist, kernel=kernel), range(Npars))
    accepted_distances = [result[1] for result in results]
    evaluated_distances = [res for result in results for res in result[3]]  # flatten list
    newparlist = [result[0] for result in results]
    newweights = [result[2] for result in results]

    newweights /= np.sum(newweights)  # Normalizing
    print("acceptance rate:", Npars / len(evaluated_distances))
    print("min accepted distance: ", np.min(accepted_distances))
    print("median accepted distance: ", np.median(accepted_distances))
    print("median evaluated distance: ", np.median(evaluated_distances))

    return (newparlist, newweights, accepted_distances, Npars / len(evaluated_distances))


def Sequential_ABC(initial_dist=10000, final_dist=100, Npars=1000, prior_label=None,
                   adaptative_kernel=False):
    ## Sequence of acceptance threshold start with initial_dis and keeps on reducing until
    ## a final threshold final_dist is reached.

    ## prior_label can be used to restart a sampling from a previous prior distribution in case
    ## further exploration with a lower epsilon is needed

    distance = initial_dist
    not_converged = True
    last_round = False
    kernelfactor = 1.0

    if prior_label is None:
        pars = None
        weights = None
        idistance = 0
    else:  # a file with the label is used to load the posterior, use always numerical (not 'final')
        pars = np.loadtxt('smc/pars_{}_{}_{}_{}.out'.format(model, sto, gamma, prior_label))
        weights = np.loadtxt('smc/weights_{}_{}_{}_{}.out'.format(model, sto, gamma, prior_label))
        accepted_distances = np.loadtxt('smc/distances_{}_{}_{}_{}.out'.format(model, sto, gamma, prior_label))
        distance = np.median(accepted_distances)
        idistance = prior_label

    while not_converged:

        idistance += 1
        print("SMC step with target distance: {}".format(distance))
        pars, weights, accepted_distances, acceptance = GeneratePars(previousparlist=pars,
                                                                     previousweights=weights, eps_dist=distance,
                                                                     Npars=Npars, kernelfactor=kernelfactor)
        proposed_dist = np.median(accepted_distances)
        if last_round is True:
            not_converged = False
            label = 'final'
        else:
            label = idistance
        if proposed_dist < final_dist:
            distance = final_dist
            last_round = True
        else:
            distance = proposed_dist
        np.savetxt('smc/pars_{}.out'.format(label), pars)
        np.savetxt('smc/weights_{}.out'.format(label), weights)
        np.savetxt('smc/distances_{}.out'.format(label), accepted_distances)

        if acceptance < 0.1 and kernelfactor > 0.1 and adaptative_kernel:
            kernelfactor = kernelfactor * 0.7
            print('Reducing kernel width to : ', kernelfactor)
        elif acceptance > 0.5 and kernelfactor < 1 and adaptative_kernel:
            kernelfactor = kernelfactor / 0.7
            print('Increasing kernel width to : ', kernelfactor)

    print('ABC converged succesfully!\n')



SyntaxError: ignored

In [106]:
def distance(par):  
    ### This function defines the distance between the data and the model for a given set of parameters
    ### for instance it can be a least squares minimization e.g. for a set of experimental observations
    ### stored as xdata and ydata:
    par = (tuple(par),)
    traj = get_trajectory(timespan, initcond, t_eval, par)
    print(traj)
    print(c_data, traj.y[0])
    print(s_data, traj.y[1])
    difference = np.sum(np.power(c_data - traj.y[0], 2) + np.power(s_data - traj.y[1], 2))
    return difference



In [107]:
processcall=0
previousparlist=None
previousweights=None
eps_dist=1000000000000
kernel=None

#seed()  # setting random seeds for each thread/process to avoid having the same random sequence in each thread
#np.random.seed()

evaluated_distances = []
d = eps_dist + 1  # original distance is beyond eps_dist
while d > eps_dist:  # until a suitable parameter is found
     if previousparlist is None:  # if is the first iteration
         proposed_pars = sampleprior()
     else:
         print('+ch', end='')
         selected_pars = choices(previousparlist, weights=previousweights, k=1)  # select a point from the previous sample
         print('-ch', end='')
         proposed_pars = selected_pars + kernel.rvs()  # and perturb it
     
     if (evaluateprior(proposed_pars, model) > 0):
         print('+epd', proposed_pars[0], end=' ')
         d = distance(proposed_pars)
         if d is None:
           break
         evaluated_distances.append(d)
         print('distance:', d)

# Calculate weight
if previousparlist is None:
    weight = 1
else:
    sum_denom = 0
    for ipars, pars in enumerate(previousparlist):
        kernel_evaluation = kernel.pdf(proposed_pars - pars)
        sum_denom += kernel_evaluation * previousweights[ipars]
    weight = evaluateprior(proposed_pars) / sum_denom
    
print('end of GeneratePar')

Streaming output truncated to the last 5000 lines.
     njev: 0
      nlu: 0
      sol: None
   status: 0
  success: True
        t: array([ 0,  2,  4, 24])
 t_events: None
        y: array([[ 2.71666666e+02,  3.46724391e+02,  3.46724391e+02,
         3.46724391e+02],
       [ 4.16666660e+01,  5.04624025e+01,  5.04624025e+01,
         5.04624025e+01],
       [ 2.00000000e+03, -1.35346208e-06, -2.19083410e-07,
         2.58944145e-07],
       [ 1.00000000e+02,  2.07559854e-36,  1.29816784e-73,
         0.00000000e+00],
       [ 1.00000000e+00,  1.12840165e+00,  1.10017168e+00,
         8.53941134e-01]])
 y_events: None
[4.16666667e+01 4.51666667e+03 2.29000000e+07 1.79333333e+08] [271.666666   346.72439061 346.72439056 346.72439054]
[2.71666667e+04 1.20000000e+03 1.76666667e+06 2.26666667e+07] [41.666666   50.46240253 50.46240253 50.46240253]
distance: 3.3201611383719224e+16
+epd 0.7266215815312616   message: 'The solver successfully reached the end of the integration interval.'
     nf

KeyboardInterrupt: ignored

In [80]:
print(d)

None


In [88]:
def get_trajectory(timespan, initcond, t_eval, pars):
    #print('get_traj pars:', pars)
    #print('\t+ode', end='')
    traj = solve_ivp(model, t_span=timespan, y0=initcond, t_eval=t_eval, args=pars)
    #print('-ode', end='\t')
    return traj


In [100]:
x_data = np.array([0, 2,4,24])  # input data, time points
#c_data = np.array([0, 883, 6370, 1360000])*(10**6) # input data, population/10^6
#s_data = np.array([0, 550, 1970, 91700])*(10**6) # input data, population/10^6
s_data = np.array([27166666667,1200000000.00,1766666666666.67,22666666666666.70])/1000000
c_data = np.array([41666666.67,4516666666.67,22900000000000.00,179333333333333.00])/1000000

initcond = [271.666666,41.666666,2000,100,1]

get_trajectory(timespan, initcond, x_data, (tuple(proposed_pars),))

  message: 'The solver successfully reached the end of the integration interval.'
     nfev: 7052
     njev: 0
      nlu: 0
      sol: None
   status: 0
  success: True
        t: array([ 0,  2,  4, 24])
 t_events: None
        y: array([[2.71666666e+02, 3.76652540e+02, 3.76652540e+02, 3.76652540e+02],
       [4.16666660e+01, 5.33824494e+01, 5.33824494e+01, 5.33824495e+01],
       [2.00000000e+03, 8.61087253e-07, 7.94489635e-07, 5.03019226e-07],
       [1.00000000e+02, 1.03974317e-38, 2.13795563e-79, 0.00000000e+00],
       [1.00000000e+00, 1.24607312e+00, 1.20263938e+00, 8.43440321e-01]])
 y_events: None

In [101]:


seed()  # setting random seeds for each thread/process to avoid having the same random sequence in each thread
np.random.seed()

x_data = np.array([0, 2, 4, 24])  # input data, time points
#c_data = np.array([0, 883, 6370, 1360000])*(10**6) # input data, population/10^6
#s_data = np.array([0, 550, 1970, 91700])*(10**6) # input data, population/10^6
#s_data = np.array([271666666.67,1200000000.00,1766666666666.67,22666666666666.70])
#c_data = np.array([41666666.67,4516666666.67,22900000000000.00,179333333333333.00])

#initcond = [c_data[0],s_data[0],2000,100,1]


evaluated_distances = []
proposed_pars = sampleprior()
#selected_pars = choices(previousparlist, weights=previousweights, k=1)  # select a point from the previous sample
#proposed_pars = selected_pars + kernel.rvs()  # and perturb it

evaluateprior(proposed_pars, model)
d = distance(proposed_pars)
evaluated_distances.append(d)


In [108]:
distance(proposed_pars)

  message: 'The solver successfully reached the end of the integration interval.'
     nfev: 8444
     njev: 0
      nlu: 0
      sol: None
   status: 0
  success: True
        t: array([ 0,  2,  4, 24])
 t_events: None
        y: array([[2.71666666e+002, 2.93553986e+002, 2.93553986e+002,
        2.93553986e+002],
       [4.16666660e+001, 4.81057494e+001, 4.81057494e+001,
        4.81057494e+001],
       [2.00000000e+003, 1.10688649e-006, 1.21248225e-007,
        1.13697435e-006],
       [1.00000000e+002, 1.59313437e-025, 7.20712001e-052,
        2.67169972e-279],
       [1.00000000e+000, 1.08648982e+000, 1.05722425e+000,
        8.04602980e-001]])
 y_events: None
[4.16666667e+01 4.51666667e+03 2.29000000e+07 1.79333333e+08] [271.666666   293.55398566 293.55398567 293.55398566]
[2.71666667e+04 1.20000000e+03 1.76666667e+06 2.26666667e+07] [41.666666   48.10574941 48.10574942 48.10574941]


3.3201633004920564e+16

In [103]:
print(d)

None


In [83]:
#for time interval (0-24h), initial conditions, timepoints for evaluation, and parameters
get_trajectory(timespan, initcond, t_eval, pars=((1,)*12,))  
print('MODEL PREDICTIONS OF SYSTEM: \n', traj)

KeyboardInterrupt: ignored

In [ ]:
GeneratePar( eps_dist=10^13)

+epd 0.30975496777269645 -epd: 1857978367662.9756
+epd 0.3688148271227508 -epd: 1857980577190.716
+epd 0.6036477158975444 -epd: 1857951717209.0007
+epd 0.3434252504097931 -epd: 1857969339238.8418
+epd 0.2913419077628685 -epd: 1857978676597.8193
+epd 0.7970935283946461 -epd: 1857917314976.5059
+epd 0.43616617638570737 -epd: 1857977759708.0186
+epd 0.7590532807498891 -epd: 1857907669847.4797
+epd 0.45923018245094177 -epd: 1857955643379.723
+epd 0.6199385057861203 -epd: 1857911378992.394
+epd 0.5864611331339326 -epd: 1857937984050.0
+epd 0.2571897026489628 -epd: 1857995507673.3462
+epd 0.4293866907755375 -epd: 1857944787165.331
+epd 0.3645724473245885 -epd: 1857967942683.834
+epd 0.23121073702887243 -epd: 1857992920401.2458
+epd 0.5478606252747205 -epd: 1857859138324.538
+epd 0.345810051724887 -epd: 1857969482273.3267
+epd 0.7298296161488016 -epd: 1857749910698.9197
+epd 0.6194545952820008 -epd: 1857909222149.4062
+epd 0.41679831472931944 -epd: 1857960409612.2651
+epd 0.33824045149661497 

KeyboardInterrupt: ignored

In [ ]:
GeneratePars(eps_dist=1000000000000000000000000)

  0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

Solving the ODEs

In [ ]:

def pend(y, t, growthc, growths, dc, ds, KG, KA, hydrolysis, da, de, uptake, secretion):
    #let y be a vector of
    cheater,secretor,glucose,ampicillin,enzyme = y 
    #set all 5 equations
    dydt = [growthc*(glucose/(KG+glucose))*cheater - dc*ampicillin/(KA+ampicillin),
            growths*(glucose/(KG+glucose))*secretor - ds*ampicillin/(KA+ampicillin),
            - uptake*(glucose/(KG+glucose))*(cheater+secretor),
            - hydrolysis*ampicillin*enzyme - da*ampicillin,
            secretion*(glucose/(KG+glucose))*secretor - de*enzyme]
    return dydt

#set constant parameters
growthc = 0.2
growths = 0.4
dc = 0.0
ds = 0.0
KG = 1
KA = 1
hydrolysis = 1
da = 1
de = 1
uptake = 1
secretion = 1



#set initial conditions
y0 = [1, 1, 20, 1, 1]
#ODE will be solved for 0 <= x <= 10 at 101 uniform intervals 
t = np.linspace(0, 10, 101)

#solve the system of ODEs for the given time and parameters

sol = odeint(pend, y0, t, args=(growthc, growths, dc, ds, KG, KA, hydrolysis, da, de, uptake, secretion))

#plot the solutions
import matplotlib.pyplot as plt
plt.plot(t, sol[:, 0], 'b', label='cheater(t)')
plt.plot(t, sol[:, 1], 'g', label='secretor(t)')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()